In [ ]:
import pandas as pd
from pyprojroot import here
import numpy as np
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
from colors import *

In [ ]:
fig, ax = plt.subplots(figsize=(4,4))

auc_values = {}

for path in (here() / 'data/processed/matrices/prob').glob('*-12-*.csv'):
    
    TARGET = path.stem.split('-')[0]
    MODEL = path.stem.split('-')[1]
    FS = path.stem.split('-')[2]
    HPO = path.stem.split('-')[3]
    FSE = path.stem.split('-')[4]

    if MODEL=='guess' or TARGET=='cri':
        continue
    
    pred_name = f'{TARGET}-{MODEL}-{FS}-{HPO}-{FSE}'
    true_name = f'{TARGET}'
    
    y_true = pd.read_csv(here() / f'data/processed/true_matrices/{true_name}.csv', index_col='Datetime')
    y_pred = pd.read_csv(here() / f'data/processed/matrices/prob/{pred_name}.csv', index_col='Datetime')

    df = pd.concat([y_true, y_pred], axis=1).dropna()
    y_true = df.iloc[:,0]
    y_pred = df.iloc[:,1]

    scores = []
    x = np.linspace(0,1,100)
    for i in x:
        y_thr = y_pred > i
        f1 = f1_score(y_true, y_thr)
        scores.append(f1)
    
    ax.plot(x, scores, label=f"{TARGET}-{FS}", color=c[TARGET])
    ax.set_xlim(0,1)
    ax.set_ylim(0,1)
    ax.legend(frameon=False);
    #ax.legend(frameon=False)
    ax.axvline(0.5, ls='--', c='k', lw=.5)
    #ax.axhline(0.7, ls='--', c='k', lw=.5)
    ident = [0.0, 1.0]
    
    ax.set_xlabel('Threshold')
    ax.set_ylabel('F1 score')
    
    ax.grid(ls='--', lw=.5)
    ax.set_axisbelow(True)
    #ax.plot(ident,ident, ls='--', c='k', zorder=0, lw=.7)

plt.savefig(here() / 'output/plots/f1.png', dpi=300, bbox_inches='tight')